Imports


In [1]:
import gc
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
import dask.dataframe as dd
from dask.distributed import Client

client = Client(n_workers = 1)
print(f"Dashboard Available at:{client.dashboard_link}")

warnings.simplefilter(action='ignore', category=FutureWarning)

Dashboard Available at:http://127.0.0.1:8787/status


In [2]:
def compute_and_continue(ddf):
    ddf = ddf.compute()
    ddf = dd.from_pandas(ddf,10)
    return ddf

In [3]:
def downscale_and_save(df, filename):
    df_copy = df.copy()

    for col in df_copy.select_dtypes(include='float64').columns:
        df_copy[col] = df_copy[col].astype('float32')

    df_copy.to_csv(filename, index=False)

    del df_copy
    gc.collect()

In [4]:
train_df = dd.read_csv('data/train_baby.csv', parse_dates=['date_time'])
test_df = dd.read_csv('data/test_baby.csv', parse_dates=['date_time'])

In [5]:
# Creating the relevance target
train_df['relevance'] = train_df['booking_bool'] * 2 + (train_df['click_bool'] * (1 - train_df['booking_bool']))

Handling Datetime

In [6]:
# Extract useful features from 'date_time'
train_df['year'] = train_df['date_time'].dt.year
train_df['month'] = train_df['date_time'].dt.month
train_df['day'] = train_df['date_time'].dt.day
train_df = train_df.drop(columns=['date_time'])

test_df['year'] = test_df['date_time'].dt.year
test_df['month'] = test_df['date_time'].dt.month
test_df['day'] = test_df['date_time'].dt.day
test_df = test_df.drop(columns=['date_time'])

Removing Outliers

In [7]:
num_feats_with_outliers = ['price_usd', 'comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff']

for feature in num_feats_with_outliers:  # Based on EDA only price_usd & compX_rate_percent_diff
    Q1 = train_df[feature].quantile(0.25)
    Q3 = train_df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    
    # Replace outliers with NaN
    train_df[feature].mask(~train_df[feature].between(lower_bound, upper_bound), np.nan)

Selecting Subset of Records

In [8]:
# Calculate the count of missing values in each row
train_df['missing_count'] = train_df.isnull().sum(axis=1)
# Sort the dataframe by 'missing_count' in ascending order
train_df = train_df.sort_values(by='missing_count')
# Select the top x% of the rows with the least missing values
top_percentage = 0.75
cut_off = int(len(train_df) * top_percentage)
train_df = train_df.head(cut_off)
train_df = dd.from_pandas(train_df,10)

Feature Engineering

Mean Position

In [9]:
mean_positions = train_df[train_df['random_bool'] == False].groupby('prop_id')['position'].mean().rename('mean_train_position')  # Exclude records where the results order is random
train_df = train_df.join(mean_positions, on='prop_id')
test_df = test_df.join(mean_positions, on='prop_id')

train_df = train_df.compute()
test_df = compute_and_continue(test_df)


Click/Booking Prop

In [10]:
def compute_prior(df, group_field, value_field):
    # Sum and count values per group
    #DASK sums = df.groupby(group_field)[value_field].transform('sum', meta = {'sum': int})
    sums = df.groupby(group_field)[value_field].transform('sum')
    #DASK count = df.groupby(group_field)[value_field].transform('count',  meta = {'count': int})
    count = df.groupby(group_field)[value_field].transform('count')
    # Calculate leave-one-out prior
    
    prior = (sums - df[value_field]) / (count - 1)
    
    return prior

In [11]:
# Apply function for click and booking bool
train_df['click_prior'] = compute_prior(train_df, 'prop_id', 'click_bool')
train_df['booking_prior'] = compute_prior(train_df, 'prop_id', 'booking_bool')

# Handling cases with only one record per group
train_df = train_df.fillna({'click_prior': train_df['click_bool'].mean()})
train_df = train_df.fillna({'booking_prior': train_df['booking_bool'].mean()})

train_df = dd.from_pandas(train_df,10)


In [12]:
# Priors for click and booking bool from the training set
test_df['click_prior'] = test_df['prop_id'].map(train_df.groupby('prop_id')['click_bool'].mean())
test_df['booking_prior'] = test_df['prop_id'].map(train_df.groupby('prop_id')['booking_bool'].mean())

# Handling cases with only one record per group
test_df = test_df.fillna({'click_prior': train_df['click_bool'].mean()})
test_df = test_df.fillna({'booking_prior': train_df['booking_bool'].mean()})

test_df = compute_and_continue(test_df)


c:\Users\jaime\Desktop\DistributedSystems\ExternalScheduler\.venv\Lib\site-packages\dask_expr\_collection.py:4219: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  warnings.warn(meta_warning(meta))
c:\Users\jaime\Desktop\DistributedSystems\ExternalScheduler\.venv\Lib\site-packages\dask_expr\_collection.py:4219: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  warnings.warn(meta_warning(meta))


2025-01-12 16:52:25,893 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle bc3bbbfe9a8d52686abcd22dd570048c initialized by task ('shuffle-transfer-bc3bbbfe9a8d52686abcd22dd570048c', 9) executed on worker tcp://127.0.0.1:53654
2025-01-12 16:52:25,908 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f57f14f0acb6b95bbfea48e4f00a35ab initialized by task ('shuffle-transfer-f57f14f0acb6b95bbfea48e4f00a35ab', 9) executed on worker tcp://127.0.0.1:53654
2025-01-12 16:52:26,162 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 9130f2095ff82e18a4edd1403705ebd4 initialized by task ('shuffle-transfer-9130f2095ff82e18a4edd1403705ebd4', 0) executed on worker tcp://127.0.0.1:53654
2025-01-12 16:52:26,240 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 5177e8ce70b6246efb0c33eff5bf8057 initialized by task ('shuffle-transfer-5177e8ce70b6246efb0c33eff5bf8057', 0) executed on worker tcp://127.0.0.1:53654
2025-01-12 16:52:26,350 - distributed.shuffle._scheduler_plu

Number of Previous searches

In [13]:
# Number of occurences "minus the current row"
train_df['previous_searches'] = train_df.groupby('prop_id')['prop_id'].transform('count') - 1
test_df['previous_searches'] = test_df['prop_id'].map(train_df['prop_id'].value_counts() - 1).fillna(0)

train_df = compute_and_continue(train_df)
test_df = compute_and_continue(test_df)

C:\Users\jaime\AppData\Local\Temp\ipykernel_10788\3533700188.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  train_df['previous_searches'] = train_df.groupby('prop_id')['prop_id'].transform('count') - 1
c:\Users\jaime\Desktop\DistributedSystems\ExternalScheduler\.venv\Lib\site-packages\dask_expr\_collection.py:4219: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  warnings.warn(meta_warning(meta))


Number of Bookings for Property/Destination Combination

In [14]:
# Aggregate number of bookings for each property and destination combination
booking_counts = train_df.groupby(['prop_id', 'srch_destination_id'])['booking_bool'].sum().reset_index()
booking_counts = booking_counts.rename(columns={'booking_bool': 'booking_count'})

# Merge this count back to the train and test datasets
train_df = train_df.merge(booking_counts, on=['prop_id', 'srch_destination_id'], how='left')
test_df = test_df.merge(booking_counts, on=['prop_id', 'srch_destination_id'], how='left')

train_df = train_df.compute()
test_df = compute_and_continue(test_df)

Mean Distance to Other Properties in the Query

In [15]:

# Calculate the maximum difference in distance to the user within each search query
train_df['max_distance_diff'] = train_df.groupby('srch_id')['orig_destination_distance'].transform(lambda x: x.max() - x.min())

train_df = dd.from_pandas(train_df,10)
# Compute the mean of these maximum differences by property and add it back to the dataset
mean_distance = train_df.groupby('prop_id')['max_distance_diff'].mean().reset_index()
mean_distance = mean_distance.rename(columns={'max_distance_diff': 'mean_max_distance_diff'})

train_df = train_df.merge(mean_distance, on='prop_id', how='left')
test_df = test_df.merge(mean_distance, on='prop_id', how='left')

train_df = compute_and_continue(train_df)
test_df = compute_and_continue(test_df)

Statistical Features

In [16]:
features_to_stat = ['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'price_usd', 'orig_destination_distance', 'srch_query_affinity_score', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count']  # Perhaps change this based on LightGBM.feature_importances_
for feature in tqdm(features_to_stat):
    feature_groupby = train_df.groupby('prop_id')[feature]

    stats_mean = feature_groupby.agg(['mean']).rename(columns={'mean': f'{feature}_mean'})
    train_df = train_df.join(stats_mean, on='prop_id')
    
    stats_std = feature_groupby.agg(['std']).rename(columns={'std': f'{feature}_std'})
    train_df = train_df.join(stats_std, on='prop_id')

    train_df = compute_and_continue(train_df)


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:08<00:00,  1.77it/s]


In [17]:
for feature in tqdm(features_to_stat):
    feature_groupby = train_df.groupby('prop_id')[feature]

    stats_mean = feature_groupby.agg(['mean']).rename(columns={'mean': f'{feature}_mean'})
    test_df= test_df.join(stats_mean, on='prop_id')
    
    stats_std = feature_groupby.agg(['std']).rename(columns={'std': f'{feature}_std'})
    test_df = test_df.join(stats_std, on='prop_id')

    test_df = compute_and_continue(test_df)

100%|██████████| 15/15 [00:13<00:00,  1.15it/s]


Start Computation

In [18]:
train_df = train_df.compute()
test_df = test_df.compute()

See results

In [19]:
train_df

,Unnamed: 0,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,srch_length_of_stay_mean,srch_length_of_stay_std,srch_booking_window_mean,srch_booking_window_std,srch_adults_count_mean,srch_adults_count_std,srch_children_count_mean,srch_children_count_std,srch_room_count_mean,srch_room_count_std
0,0,1,12,187,NaN,NaN,219,893,3,3.5,...,1.0,NaN,0.0,NaN,4.0,NaN,0.0,NaN,1.0,NaN
0,485,44,15,55,NaN,NaN,55,74617,4,4.5,...,1.0,NaN,1.0,NaN,1.0,NaN,0.0,NaN,1.0,NaN
0,230,25,5,219,NaN,NaN,219,91587,3,4.5,...,1.0,NaN,0.0,NaN,2.0,NaN,0.0,NaN,1.0,NaN
0,756,64,5,219,NaN,NaN,219,32917,4,4.5,...,1.0,NaN,2.0,NaN,2.0,NaN,0.0,NaN,1.0,NaN
0,291,30,24,216,NaN,NaN,219,13878,3,4.0,...,9.0,NaN,13.0,NaN,2.0,NaN,0.0,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,1528,107,5,219,NaN,NaN,219,47283,3,5.0,...,1.0,NaN,5.0,NaN,1.0,NaN,0.0,NaN,1.0,NaN
149,378,39,5,219,NaN,NaN,219,46465,3,4.0,...,1.0,NaN,7.0,NaN,2.0,NaN,2.0,NaN,1.0,NaN
149,214,25,5,219,NaN,NaN,219,53298,2,3.5,...,1.0,NaN,0.0,NaN,2.0,NaN,0.0,NaN,1.0,NaN
149,634,52,16,31,NaN,NaN,215,87428,4,4.5,...,2.0,NaN,7.0,NaN,1.0,NaN,0.0,NaN,1.0,NaN


In [20]:
test_df

,Unnamed: 0,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,srch_length_of_stay_mean,srch_length_of_stay_std,srch_booking_window_mean,srch_booking_window_std,srch_adults_count_mean,srch_adults_count_std,srch_children_count_mean,srch_children_count_std,srch_room_count_mean,srch_room_count_std
0,1180,77,5,219,NaN,NaN,219,113370,2,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1260,83,5,219,NaN,NaN,219,28460,2,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,720,51,5,219,NaN,NaN,219,85599,3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1340,87,5,219,NaN,NaN,220,56808,4,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1800,113,22,92,NaN,NaN,219,6652,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,699,51,5,219,NaN,NaN,219,2543,3,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,979,65,5,219,NaN,NaN,219,120549,3,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,199,13,15,55,3.14,107.25,55,102855,3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,1039,68,5,219,NaN,NaN,127,101342,3,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
downscale_and_save(train_df, 'data/processed_train.csv')
downscale_and_save(test_df, 'data/processed_test.csv')

#MODEL